In [1]:
import numpy as np
import time
import math
from PIL import Image, ImageDraw

In [1]:
class other_class:
    def open_image(image_path):
        image = Image.open(image_path)
        return image
    
    def mesh_search(image_array_mesh, image):
        '''
        如果(1920, 1080), n=500 ==> x_scale=4, y_scale=3
        '''
        w,h = image.size
        a,b,c,d,e = image_array_mesh.shape
        x_scale = 0
        y_scale = 0
        if c > w%c > 0:
            x_scale = (w//c)+1
        else:
            x_scale = w//c
        if c > h%c > 0:
            y_scale = (h//c)+1
        else:
            y_scale = h//c 
        print("x_scale:",x_scale)
        print("y_scale:",y_scale)
        return x_scale, y_scale
    
    def to_mesh(image_array, n=11):
        '''
        功能 : 先將圖片轉成大mesh
        內容 : 這裡用的方法是以整塊的mesh為單位來移動，最後呈現的圖像會一格格的感覺。
        ''' 
        image_array_mesh = []

        h,w,r = image_array.shape
        count_x = 0
        count_y = 0

        h1,w1 = 0, 0
        #先根據 n去劃分mesh
        for y in np.arange(0, h, n):
            count_x = 0
            count_y += n
            for x in np.arange(0, w, n):
                h1 = int((x+n)/n-1)
                w1 = int((y+n)/n-1)
                if (w-count_x) > n and (h-count_y+n) > n:
                    image_array_mesh.append(image_array[y:y+n, x:x+n])
                elif n > (w-count_x) > 0 and (h-count_y+n) > n:
                    image_array_mesh.append(image_array[y:y+n, w-n:w])
                elif     (w-count_x) > n and n > (h-count_y+n) > 0:
                    image_array_mesh.append(image_array[h-n:h, x:x+n])
                elif n > (w-count_x) > 0 and n > (h-count_y+n) > 0:
                    image_array_mesh.append(image_array[h-n:h, w-n:w])
                else:
                    print('X')
                    pass
                count_x += n
        image_array_mesh = np.array(image_array_mesh)
        image_array_mesh = image_array_mesh.reshape(w1+1, h1+1, n ,n ,3)
        return  image_array_mesh
    
    
    def mesh_to_array(image_array_mesh, image, n=11):
        '''
        image_array_mesh 的 shape為 (y_scale, x_scale, n, n,3)
        image為輸入的圖片，提供圖像大小來給定 空陣列的shape
        n為奇數，為mesh的大小
        '''
        a,b,c = np.array(image).shape
        z = np.zeros((a,b,c), dtype='uint8')
        h, w, r = z.shape
        count_x1 = 0
        count_y1 = 0
        for y in np.arange(0, h, n):
            count_x1 = 0
            count_y1 += n
            for x in np.arange(0, w, n):
                h1 = int((x+n)/n-1)
                w1 = int((y+n)/n-1)
                if   (w-count_x1) > n and (h-count_y1+n) >n:
                    z[y:y+n, x:x+n] = image_array_mesh[w1, h1]
                elif  n > (w-count_x1) > 0 and (h-count_y1+n) >n:
                    z[y:y+n, (w-n):w] = image_array_mesh[w1, h1]
                elif (w-count_x1) > n and n > (h-count_y1+n) > 0:
                    z[(h-n):h, x:x+n] = image_array_mesh[w1, h1]
                elif n > (w-count_x1) >0 and n > (h-count_y1+n) > 0:
                    z[(h-n):h, (w-n):w] = image_array_mesh[w1, h1]
                else:
                    print('X')
                    pass
                count_x1 += n
        return z
    
    def cut_image(image, w_min=0, w_max=10000, h_min=0, h_max=10000):
        print(f'影像大小 : {image.size}')
        image_array = np.array(image)
        image_array = image_array[h_min:h_max ,w_min:w_max]
        return image_array